In [170]:
!pip install langchain langchain-community langchain langchain-openai langchain-anthropic chromadb gitpython

# 코드 리뷰 LLM 서비스
---

# 1. 환경 변수 로드

In [171]:
from dotenv import load_dotenv
import os

load_dotenv()

class Config:
    OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
    ANTHROPIC_API_KEY = os.environ['ANTHROPIC_API_KEY']
    PINECONE_API_KEY = os.environ['PINECONE_API_KEY']
    CHROMA_DB_PATH = os.environ['CHROMA_DB_PATH']

---
# 2. LLM(OpenAI, Anthropic)

In [172]:
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic

openai_llm = ChatOpenAI(model="gpt-4o-mini", api_key=Config.OPENAI_API_KEY)
anthropic_llm = ChatAnthropic(model_name="claude-3-5-sonnet-20241022", api_key=Config.ANTHROPIC_API_KEY)

---
# 3. Embedding(OpenAI)

In [173]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(openai_api_key=Config.OPENAI_API_KEY)

---
# 4. VectorStore(ChromaDB)

In [174]:
from langchain.vectorstores import Chroma

vector_store = Chroma(persist_directory=Config.CHROMA_DB_PATH, embedding_function=embeddings)

---
# 5. Github repository -> Store

In [175]:
from git import Repo
import hashlib

# GitHub 레포지토리를 로컬로 클론
def clone_github_repo(repo_url, local_path):
    if os.path.exists(local_path):
        print("로컬에 이미 레포지토리가 존재합니다. 변경 사항을 pull합니다.")
        repo = Repo(local_path)
        repo.remotes.origin.pull()
    else:
        print(f"Cloning {repo_url} into {local_path}...")
        Repo.clone_from(repo_url, local_path)
    print("레포지토리 클론 또는 업데이트 완료!")

# 레포지토리 내 Python 파일 경로를 찾기
def find_python_files(repo_path):
    python_files = []
    for root, _, files in os.walk(repo_path):
        for file in files:
            if file.endswith(".py"):
                python_files.append(os.path.join(root, file))
    return python_files

# 파일의 해시값(SHA256)을 계산
def calculate_file_hash(file_path):
    sha256 = hashlib.sha256()
    with open(file_path, "rb") as f:
        while chunk := f.read(8192):
            sha256.update(chunk)
    return sha256.hexdigest()

# 파일이 업데이트되었는지 확인
def is_file_updated(file_path, vector_store):
    file_hash = calculate_file_hash(file_path)
    # Chroma에서 해당 파일의 메타데이터 검색
    results = vector_store.similarity_search(file_path, k=1, filter={"file_path": file_path})
    if results:
        # 저장된 해시값과 비교
        return results[0].metadata.get("file_hash") != file_hash
    return True

# Python 파일을 읽어 벡터화하고 Chroma DB에 저장
def store_python_files_in_chroma(file_paths, vector_store):
    for file_path in file_paths:
        try:
            if not is_file_updated(file_path, vector_store):
                print(f"파일이 변경되지 않았습니다. 건너뜁니다: {file_path}")
                continue

            with open(file_path, "r", encoding="utf-8") as file:
                code_content = file.read()

            # 파일 메타데이터 생성
            metadata = {
                "file_path": file_path,
                "file_hash": calculate_file_hash(file_path)
            }

            # 벡터 스토어에 추가
            vector_store.add_texts([code_content], metadatas=[metadata])
            print(f"{file_path} 저장 완료!")
        except Exception as e:
            print(f"파일 처리 중 오류 발생: {file_path}, 오류: {e}")
    vector_store.persist()
    print("Chroma DB에 모든 파일 저장 완료!")

# Chroma DB에 저장된 모든 코드 파일의 메타데이터와 내용을 출력합니다.
def list_all_files_with_content(vector_store):
    try:
        results = vector_store.get(include=["metadatas", "documents"])

        print("저장된 코드 파일과 내용:")
        for i, (metadata, document) in enumerate(zip(results["metadatas"], results["documents"]), 1):
            file_path = metadata.get("file_path", "Unknown")
            print(f"{i}. 파일 경로: {file_path}")
            print("내용:")
            print(document[:300])  # 첫 300자만 출력
            print("-" * 50)
    except Exception as e:
        print(f"전체 파일 목록 및 내용 출력 중 오류 발생: {e}")

# 파일명을 기준으로 Chroma DB에서 내용을 검색하고 반환.
def search_by_filename(file_name, vector_store):
    try:
        # Chroma DB의 모든 데이터를 가져옵니다.
        results = vector_store.get(include=["metadatas", "documents"])

        matching_files = []
        for metadata, document in zip(results["metadatas"], results["documents"]):
            if file_name in metadata.get("file_path", ""):
                matching_files.append((metadata, document))

        if matching_files:
            return matching_files[0][1]
        else:
            print(f"파일명 '{file_name}'이(가) DB에 존재하지 않습니다.")
            return None
    except Exception as e:
        print(f"파일명으로 검색 중 오류 발생: {e}")

In [176]:
github_repo_url = 'https://github.com/adunStudio/codereviewexample'
local_repo_path = "./temp_repo"

# 1. GitHub 레포지토리 클론 또는 업데이트
clone_github_repo(github_repo_url, local_repo_path)

# 2. Python 파일 찾기
python_files = find_python_files(local_repo_path)
print(f"찾은 Python 파일 수: {len(python_files)}")

# 3. Python 파일을 Chroma DB에 저장
store_python_files_in_chroma(python_files, vector_store)

#list_all_files_with_content(vector_store)

로컬에 이미 레포지토리가 존재합니다. 변경 사항을 pull합니다.
레포지토리 클론 또는 업데이트 완료!
찾은 Python 파일 수: 8
파일이 변경되지 않았습니다. 건너뜁니다: ./temp_repo/world.py
파일이 변경되지 않았습니다. 건너뜁니다: ./temp_repo/cat.py
파일이 변경되지 않았습니다. 건너뜁니다: ./temp_repo/text_processor.py
파일이 변경되지 않았습니다. 건너뜁니다: ./temp_repo/bird.py
파일이 변경되지 않았습니다. 건너뜁니다: ./temp_repo/animal.py
파일이 변경되지 않았습니다. 건너뜁니다: ./temp_repo/calculator.py
파일이 변경되지 않았습니다. 건너뜁니다: ./temp_repo/dog.py
파일이 변경되지 않았습니다. 건너뜁니다: ./temp_repo/main.py
Chroma DB에 모든 파일 저장 완료!


---
# 6. Retriever

In [177]:
retriever = vector_store.as_retriever(search_kwargs={"k": 5})

In [178]:
input_code = search_by_filename('cat.py', vector_store)

retrievered_codes = retriever.invoke(input_code)

print("\n### 출처 코드 ###")
print(f'문서 개수: {len(retrievered_codes)}')
for code in retrievered_codes:
    print(f"파일: {code.metadata.get('file_path', 'N/A')}")
    print(f"내용: {code.page_content[:200]}")  # 문서 내용 일부 출력
    print('-------' * 5)


### 출처 코드 ###
문서 개수: 5
파일: ./temp_repo/cat.py
내용: from animal import Animal

class Cat(Animal):
    def __init__(self, name, age, color):
        super().__init__(name, age)
        self.color = color
        self.sound = "야옹"

    def scratch(self, 
-----------------------------------
파일: ./temp_repo/dog.py
내용: from animal import Animal

class Dog(Animal):
    def __init__(self, name, age, breed):
        super().__init__(name, age)
        self.breed = breed
        self.sound = "멍멍"

    def fetch(self, it
-----------------------------------
파일: ./temp_repo/animal.py
내용: class Animal:
    def __init__(self, name, age):
        self.name = name
        self.age = age  
        self.sound = None

    def make_sound(self):
        print(f"{self.name}이(가) 소리를 냅니다: {self.s
-----------------------------------
파일: ./temp_repo/bird.py
내용: from animal import Animal

class Bird(Animal):
    def __init__(self, name, age, wing_span):
        super().__init__(name, age)
        self.wing_span =

---
# 7. PromptTemplates

In [179]:
from langchain.prompts import PromptTemplate
from langchain.prompts import load_prompt

simple_review_prompt = load_prompt('./prompts/simple_review_template.json')

input_message = ''

for code in retrievered_codes:
    input_message += f"\n\n[파일명: {code.metadata.get('file_path', 'N/A')}]\n\n"
    input_message += code.page_content

#review_prompt = review_template.format(context=input_message)
#print(review_prompt)

---
# 8. TransformChain(Retriever -> PromptTemplate)

In [180]:
from langchain.chains import TransformChain

def create_input_code(inputs):
    documents = inputs["documents"]
    input_message = ""
    for doc in documents:
        input_message += f"\n\n[파일명: {doc.metadata.get('file_path', 'N/A')}]\n\n"
        input_message += doc.page_content
    return {"context": input_message}

code_mapping_chain = TransformChain(
    input_variables=["documents"],
    output_variables=["context"],
    transform=create_input_code
)

---
# 9. Chain 연결

In [181]:
openai_chain    = retriever | code_mapping_chain | simple_review_prompt | openai_llm
anthropic_chain = retriever | code_mapping_chain |simple_review_prompt | anthropic_llm

# cat.py 파일과 연관된 코드들을 찾는다.
input_code = search_by_filename('cat.py', vector_store)

---
# 간단 버전
## OpenAI 응답 결과

In [182]:
from langchain.callbacks import get_openai_callback

with get_openai_callback() as callback:
    simple_openai_response = openai_chain.invoke(input_code)

    # 토큰 관련 정보 출력
    print(f"입력 토큰 수: {callback.prompt_tokens}")
    print(f"응답 토큰 수: {callback.completion_tokens}")
    print(f"총 토큰 수: {callback.total_tokens}")
    print(f"총 비용: ${callback.total_cost}")

입력 토큰 수: 919
응답 토큰 수: 688
총 토큰 수: 1607
총 비용: $0.00055065


In [183]:
print(simple_openai_response.content)

### [파일명: ./temp_repo/cat.py]

- `scratch` 메서드에서 `self.item`이 아니라 `item`을 사용해야 합니다. 잘못된 변수를 참조하고 있습니다.
- 불필요한 코드나 성능 문제는 없습니다. 그러나 "야옹"을 sound로 정의한 것은 중복성을 줄이기 위해 `make_sound` 메서드에서 호출하면 좋겠습니다. 
- 나머지 부분은 잘 작성되었습니다. 

수정된 코드:
```python
def scratch(self, item):
    if not item:
        print(f"{self.name}이(가) 긁을 것이 없습니다!")
    else:
        print(f"{self.name}이(가) {item}을(를) 긁습니다.")
```

### [파일명: ./temp_repo/dog.py]

- `fetch` 메서드 내의 for 루프는 성능 저하를 초래합니다. 쓸모없는 반복문으로 보입니다. 이를 제거할 수 있습니다.
- `fetch` 메서드는 `make_sound` 메서드를 호출하여 먼저 소리를 내게 하고, 그다음에 물건을 가져오도록 하는 것이 좋습니다.
- 나머지 부분은 잘 작성되었습니다.

수정된 코드:
```python
def fetch(self, item):
    self.make_sound()
    print(f"{self.name}이(가) {item}을(를) 가져옵니다!")
```

### [파일명: ./temp_repo/animal.py]

- 클래스 구조는 간단하고 명확하며, SOLID 원칙에 맞습니다.
- 다만, `make_sound` 메서드에서 소리가 None일 경우 예외적인 처리가 없으므로 그 경우에 대한 처리를 추가하면 좋겠습니다.

수정된 코드:
```python
def make_sound(self):
    if self.sound:
        print(f"{self.name}이(가) 소리를 냅니다: {self.sound}")
    else:
        print(f"{

---
## Anthropic 응답 결과

In [184]:
simple_anthropic_response = anthropic_chain.invoke(input_code)
print(simple_anthropic_response.content)

각 파일에 대한 리뷰를 진행하겠습니다.

cat.py:
1. scratch 메서드에서 item 매개변수와 self.item이 불일치합니다.
2. info 메서드에서 string concatenation 대신 f-string 사용을 권장합니다.

```python
def scratch(self, item):
    if not item:
        print(f"{self.name}이(가) 긁을 것이 없습니다!")
    else:
        print(f"{self.name}이(가) {item}을(를) 긁습니다.")

def info(self):
    super().info()
    print(f"색깔: {self.color}")
```

dog.py:
1. fetch 메서드의 불필요한 for 루프는 제거해야 합니다.
```python
def fetch(self, item):
    print(f"{self.name}이(가) {item}을(를) 가져옵니다!")
```

animal.py:
1. sound 속성에 대한 타입 힌트 추가를 권장합니다.
```python
from typing import Optional

class Animal:
    def __init__(self, name: str, age: int):
        self.name = name
        self.age = age
        self.sound: Optional[str] = None
```

bird.py:
1. fly 메서드에서 불필요한 문자열 연산을 제거합니다.
```python
def fly(self):
    print(f"{self.name}이(가) 날개 길이 {self.wing_span}cm로 날아오릅니다!")
```

world.py:
1. isinstance 체크보다는 다형성을 활용하는 것이 좋습니다.
2. simulate 메서드의 리팩토링이 필요합니다.

```python
class World:
    def __init__(self):
  